# Automatic generation of coupled cluster equations

In this example we use wick&d to generate arbitrary-order coupled cluster equations.

## Load wick&d

We start by importing the wick&d module and some other useful modules

In [1]:
import wicked as w
import time
from IPython.display import display, Math, Latex

def latex(expr):
    """Function to render any object that has a member latex() function"""
    display(Math(expr.latex()))

# Define the orbital spaces
Here we define two orbital spaces consistent with a single Slater determinant reference and we assign indices. We define 1) a space of occupied orbitals (`o`) and 2) a space of unoccupied orbitals (`v`)

In [2]:
w.reset_space()
w.add_space('o','fermion','occupied',['i','j','k','l','m','n'])
w.add_space('v','fermion','unoccupied',['a','b','c','d','e','f'])

## Define the Hamiltonian operator

Next, we define the components of the Hamiltonian operator

In [6]:
E0 = w.op("E_0",[''])
F = w.utils.gen_op('f',1,'ov','ov')
V = w.utils.gen_op('v',2,'ov','ov')
H = E0 + F + V

## Define a function to compute the cluster operator truncated to level *n*

Since we are interested in arbitrary-order equations, we write a simple function that generates the cluster operator for any truncation level $n$

In [8]:
def make_T(n):
    components = [f"{'v+' * k} {'o' * k}" for k in range(1,n + 1)]
    return w.op("t",components)

make_T(3)

+ t { v+ o }
+ 1/4 t { v+ v+ o o }
+ 1/36 t { v+ v+ v+ o o o }

## Setup the similarity-transformed Hamiltonian and compute expectation value

Here we define a function that defines the cluster operator, computes $\bar{H}$, and evaluates Wick's theorem.
At the end we derive many-body equations and count how many operators are there for each excitation rank.

In [9]:
def cc_equations(n):
    start = time.perf_counter()
    wt = w.WickTheorem()

    T = make_T(n)
    Hbar = w.bch_series(H,T,4)
    expr = wt.contract(w.rational(1), Hbar, 0, 2 * n)
    mbeq = expr.to_manybody_equation("r")
    end = time.perf_counter()    
    t = end - start
    
    equations = {}
    for r in range(0,n + 1):
        s = f"{'o' * r}|{'v' * r}" 
        equations[r] = (mbeq[s])      
        
    return equations, t

## Running the code

In the next line we run the code and get the timing!

In [13]:
equations, t = cc_equations(5)

# count the number of terms for each rank
s = 0
for rank,eq in equations.items():
    print(f'Rank {rank}: {len(eq)} equations')  
    s += len(eq)
    
print(f'\nGenerated {s} equations in {t:.3f} seconds')    

Rank 0: 4 equations
Rank 1: 15 equations
Rank 2: 38 equations
Rank 3: 54 equations
Rank 4: 80 equations
Rank 5: 99 equations

Generated 290 equations in 6.666 seconds


We can inspect all the singles equations

In [14]:
equations[1]

[r^{o0}_{v0} += - f^{o0}_{o1} t^{o1}_{v0},
 r^{o0}_{v0} += - f^{v1}_{o1} t^{o1}_{v0} t^{o0}_{v1},
 r^{o0}_{v0} +=  f^{v1}_{o1} t^{o0,o1}_{v0,v1},
 r^{o0}_{v0} +=  f^{o0}_{v0},
 r^{o0}_{v0} +=  f^{v1}_{v0} t^{o0}_{v1},
 r^{o0}_{v0} += - t^{o1}_{v0} t^{o0}_{v1} t^{o2}_{v2} v^{v1,v2}_{o1,o2},
 r^{o0}_{v0} += - t^{o1}_{v0} t^{o2}_{v1} v^{o0,v1}_{o1,o2},
 r^{o0}_{v0} += -1/2 t^{o1}_{v0} t^{o0,o2}_{v1,v2} v^{v1,v2}_{o1,o2},
 r^{o0}_{v0} += -1/2 t^{o1,o2}_{v0,v1} v^{o0,v1}_{o1,o2},
 r^{o0}_{v0} += 1/4 t^{o0,o1,o2}_{v0,v1,v2} v^{v1,v2}_{o1,o2},
 r^{o0}_{v0} += -1/2 t^{o0}_{v1} t^{o1,o2}_{v0,v2} v^{v1,v2}_{o1,o2},
 r^{o0}_{v0} += - t^{o0}_{v1} t^{o1}_{v2} v^{v1,v2}_{o1,v0},
 r^{o0}_{v0} +=  t^{o1}_{v1} t^{o0,o2}_{v0,v2} v^{v1,v2}_{o1,o2},
 r^{o0}_{v0} += - t^{o1}_{v1} v^{o0,v1}_{o1,v0},
 r^{o0}_{v0} += -1/2 t^{o0,o1}_{v1,v2} v^{v1,v2}_{o1,v0}]

And generate nice-looking equations

In [15]:
for eq in equations[1]:
    latex(eq)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Counting the number of diagrams

In the following we generate a table that reports the total number of coupled cluster diagrams for up to CCSDTQP

In [16]:
rows = []
for n in range(1,6):
    equations, t = cc_equations(n)
    count = " ".join([f'{len(eqs):4d}' for k, eqs in equations.items()])
    rows.append(f' {n}  {t:4.1f} {count}')

width = 39
print(f'{"=" * width}')
print(f' n  time      Excitation level ')
print(f'     (s)    0    1    2    3    4    5 ')
print(f'{"-" * width}')
print("\n".join(rows))
print(f'{"=" * width}')    

 n  time      Excitation level 
     (s)    0    1    2    3    4    5 
---------------------------------------
 1   0.0    3    8
 2   0.1    4   14   31
 3   0.7    4   15   37   47
 4   2.5    4   15   38   53   74
 5   6.5    4   15   38   54   80   99
